## Beginning of algo playground

In [1]:
import sys
sys.path.insert(0, '/Users/chingaling/Documents/Algo Trading/BroCodePC')
import IB_API_tutorial as ib
import main as m
import time
import schedule
import pandas as pd
from pandas.tseries.offsets import BDay
import numpy as np
import math
import datetime
from itertools import cycle
import pytz

## Create some global variables and open connection

In [2]:
port = 7496
ip = "127.0.0.1"
clientID = 1
buyFactor = .99
SellFactor = 1.01
rebalance_interval = 10
maxBuyOrdersAtOnce = 30
minInvSize = 1000
stock_list = []
maxCandidates=100
leastPrice=3.00
mostPrice=20.00
fireSalePrice = leastPrice
fireSaleAge = 6

# Local file addresses
csvAddress_stocks_to_trade = "/Users/chingaling/Documents/Algo Trading/BroCodePC/stocks_to_trade.csv"
csvAddress_exec = "/Users/chingaling/Documents/Algo Trading/BroCodePC/executions.csv"
csvAddress_orders = "/Users/chingaling/Documents/Algo Trading/BroCodePC/orders.csv"
pklAdd_orders = "/Users/chingaling/Documents/Algo Trading/BroCodePC/orders.pkl"
pklAdd_exec = "/Users/chingaling/Documents/Algo Trading/BroCodePC/executions.pkl"

In [ ]:
app.disconnect()
from importlib import reload
reload (ib)

In [3]:
# Test connection
#clientID = 2
app = ib.TestApp(ip, port, clientID)
current_time = app.speaking_clock()
print(current_time)

Getting the time from the server... 
IB error id -1 errorcode 2104 string Market data farm connection is OK:njfarm.nj
IB error id -1 errorcode 2104 string Market data farm connection is OK:eufarm
IB error id -1 errorcode 2104 string Market data farm connection is OK:njfarm
IB error id -1 errorcode 2106 string HMDS data farm connection is OK:ushmds.us
IB error id -1 errorcode 2106 string HMDS data farm connection is OK:ilhmds
IB error id -1 errorcode 2106 string HMDS data farm connection is OK:hkhmds
IB error id -1 errorcode 2106 string HMDS data farm connection is OK:fundfarm
IB error id -1 errorcode 2106 string HMDS data farm connection is OK:ushmds
1543448042


In [ ]:
a = pd.read_pickle(pklAdd_exec)
b = pd.read_pickle(pklAdd_orders)

In [ ]:
a.to_csv(csvAddress_exec)
b.to_csv(csvAddress_orders)

## Executing main trading algo

In [ ]:
app.main_trading()

## Making buy orders

In [ ]:
# Load the list of 100 stocks to consider trading today
app.generate_stock_shortlist()

In [ ]:
# Run algorithm to create all of the buy orders.
app.rebalance_buy_orders()

## Cancelling orders

In [ ]:
# Cancel all orders
app.cancel_all_orders()

In [ ]:
# Cancel all open buy orders
app.cancel_all_open_buy_orders(minutes=rebalance_interval)

In [4]:
# Cancel all open buy orders
app.cancel_all_open_buy_orders(minutes=0)

Cancelling this order: MFIN
298
IB error id 298 errorcode 202 string Order Canceled - reason:
Cancelling this order: TACT
299
IB error id 299 errorcode 202 string Order Canceled - reason:


## Submitting sell orders

In [ ]:
app.rebalance_all_sell_orders()

## Getting executions and commission

In [ ]:
## lets get positions
positions_list = app.get_current_positions()
print(positions_list)

In [ ]:
## get the account name from the position
## normally you would know your account name
accountName = positions_list.account[0]

In [ ]:
accountName

In [ ]:
def get_available_funds():

    '''
    Retrieves the amount of funds available to use for limit buy orders
    
    Returns: Float
    '''
    
    accounting_values = app.get_accounting_values(accountName)
    a = pd.DataFrame(accounting_values, columns=['desc', 'amount', 'curr'])
    a.set_index(keys='desc', inplace=True)
    return float(a.loc['AvailableFunds-S'].amount)

available_funds = get_available_funds()

In [ ]:
available_funds

In [ ]:
## these values are cached
## if we ask again in more than 5 minutes it will update everything
accounting_updates = app.get_accounting_updates(accountName)
print(accounting_updates)

## Playground

In [ ]:
a = app.get_current_positions()

In [ ]:
accountName

In [ ]:
positions_list = app.get_current_positions()
accountName = positions_list.account[0]
accounting_values = app.get_accounting_values(accountName)
b = pd.DataFrame(accounting_values, columns=['desc', 'amount', 'curr'])
b

In [ ]:
c = app.get_available_funds()

min_cushion = 500
d = c - min_cushion
d

min_inv_size = 1000
e = d / min_inv_size 
f = math.floor(e)

for i in range(f):
    print(i)

In [1]:
import time

def countdown(t):
    while t:
        mins, secs = divmod(t, 60)
        timeformat = '{:02d}:{:02d}'.format(mins, secs)
        print(timeformat, end='\r')
        time.sleep(1)
        t -= 1
    print('Goodbye!\n\n\n\n\n')

In [3]:
time.sleep(3)
countdown(5)

Goodbye!







## Original algo code from Quantopian

In [ ]:
# Re-balancing method from 30_stock algo every 10 minutes. Buy order cycles through list of 100 stock selected each day.

def my_rebalance(context, data):
    buyFactor=.99
    SellFactor=1.01
    cash=context.portfolio.cash

    cancel_open_buy_orders(context, data)

    # Order sell at profit target in hope that somebody actually buys it
    for stock in context.portfolio.positions:
        if not get_open_orders(stock):
            StockShares = context.portfolio.positions[stock].amount
            CurrPrice = float(data.current([stock], 'price'))
            CostBasis = float(context.portfolio.positions[stock].cost_basis)
            SellPrice = float(make_div_by_05(CostBasis*SellFactor, buy=False))
            
            
            if np.isnan(SellPrice) :
                pass # probably best to wait until nan goes away
            elif (stock in context.age and context.age[stock] == 1) :
                pass
            elif (
                stock in context.age 
                and context.MyFireSaleAge<=context.age[stock] 
                and (
                    context.MyFireSalePrice>CurrPrice
                    or CostBasis>CurrPrice
                )
            ):
                if (stock in context.age and context.age[stock] < 2) :
                    pass
                elif stock not in context.age:
                    context.age[stock] = 1
                else:
                    SellPrice = float(make_div_by_05(.95*CurrPrice, buy=False))
                    print("Selling {}, \tshares: {}, \tprice: {}".format(stock, StockShares, SellPrice))
                    order(stock, -StockShares,
                        style=LimitOrder(SellPrice)
                    )
            else:
                if (stock in context.age and context.age[stock] < 2) :
                    pass
                elif stock not in context.age:
                    context.age[stock] = 1
                else:
                    print("Selling {}, \tshares: {}, \tprice: {}".format(stock, StockShares, SellPrice))
                    order(stock, -StockShares,
                        style=LimitOrder(SellPrice)
                    )

    WeightThisBuyOrder=float(1.00/context.maxBuyOrdersAtOnce)
    for ThisBuyOrder in range(context.maxBuyOrdersAtOnce):
        stock = next(context.MyCandidate)
        PH = data.history([stock], 'price', 20, '1d')
        PH_Avg = float(PH.mean())
        CurrPrice = float(data.current([stock], 'price'))
        if np.isnan(CurrPrice):
            pass # probably best to wait until nan goes away
        else:
            if CurrPrice > float(1.25*PH_Avg):
                BuyPrice=float(CurrPrice)
            else:
                BuyPrice=float(CurrPrice*buyFactor)
            BuyPrice=float(make_div_by_05(BuyPrice, buy=True))
            StockShares = int(WeightThisBuyOrder*cash/BuyPrice)
            #print("Buying {}, \tshares: {}, \tprice: {}".format(stock, StockShares, BuyPrice))
            order(stock, StockShares,
                style=LimitOrder(BuyPrice)
            )

## Lesson 1: Create a connection with IB API
https://qoppac.blogspot.com/2017/03/interactive-brokers-native-python-api.html

## Lesson 2: Get historic data
https://qoppac.blogspot.com/2017/03/historic-data-from-native-ib-pyhon-api.html

## Lesson 3: Get streaming data
https://qoppac.blogspot.com/2017/03/streaming-market-data-from-native.html

## Lesson 4: Creating orders
https://qoppac.blogspot.com/2017/03/placing-orders-in-native-python-ib-api.html

## Lesson 5: Getting portfolio positions and account info
https://qoppac.blogspot.com/2017/03/getting-position-and-accounting-data.html